In [26]:
import requests
from bs4 import BeautifulSoup
from deep_translator import GoogleTranslator
import pandas as pd
import os

def d_info_filmes(caminho_d_filmes, caminho_saida):

    def _buscar_filme_imdb(nome_filme):
        url_busca = f"https://www.imdb.com/find?q={
            nome_filme}&s=tt&exact=false&ref_=fn_al_tt_mr"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36', 'Accept-Language': 'pt-BR,pt;q=0.9'}
        response = requests.get(url_busca, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        link = soup.find('a', class_='ipc-metadata-list-summary-item__t')
        return "https://www.imdb.com" + link['href'] if link else None, nome_filme


    def _extrair_info_filme(filme_url):
        base_url = "https://www.imdb.com"
        headers = {"User-Agent": "Mozilla/5.0"}

        busca_url = f"{base_url}/find?q={titulo.replace(' ', '+')}&s=tt&ttype=ft"
        resposta = requests.get(busca_url, headers=headers)
        soup = BeautifulSoup(resposta.text, "html.parser")

        resultado = soup.select_one("a[href^='/title/tt']")
        if not resultado:
            print(f"Nenhum resultado encontrado para '{titulo}'.")
            return None

        link_filme = base_url + resultado['href']
        resposta_filme = requests.get(link_filme, headers=headers)
        soup = BeautifulSoup(resposta_filme.text, "html.parser")

        # Extraindo gêneros
        # generos = ", ".join(g.get_text(strip=True) for g in soup.select(
        #     'span.ipc-chip__text') if g.get_text(strip=True) != ["Voltar ao início", ""]) or "N/A"
                # Extraindo gêneros únicos e válidos
        generos_lista = []
        for g in soup.select('span.ipc-chip__text'):
            texto = g.get_text(strip=True)
            if texto and texto.lower() != "voltar ao início" and texto.lower() not in [s.lower() for s in generos_lista]:
                generos_lista.append(texto)

        # Limitando a no máximo 3
        generos = ", ".join(generos_lista[:3]) if generos_lista else "N/A"



        # Avaliação IMDb
        avaliacao_element = soup.find(
            'span', class_='sc-d541859f-1 imUuxf')
        avaliacao = avaliacao_element.text if avaliacao_element else "N/A"

        # Poster
        poster_element = soup.find('div', class_='ipc-poster')
        poster = poster_element.find('img').get(
            'src') if poster_element and poster_element.find('img') else "N/A"

        # Diretores (agora pegando todos os diretores)
        bloco_direcao = soup.select_one("li[data-testid='title-pc-principal-credit']:has(span:contains('Direção'))")

        # Se não encontrar em português, tenta em inglês
        if not bloco_direcao:
            bloco_direcao = soup.select_one("li[data-testid='title-pc-principal-credit']:has(span:contains('Director'))")

        # Extraindo os nomes se o bloco for encontrado
        if bloco_direcao:
            diretores = [a.get_text(strip=True) for a in bloco_direcao.select("ul li a")]
        else:
            diretores = ["N/A"]

        # Garantindo resultado mesmo se vazio
        diretores = diretores if diretores else ["N/A"]
        diretores = ", ".join(diretores)





        # Seleciona os elementos de empresa na página
        empresas_elementos = soup.select(
            'li.ipc-metadata-list__item:has(a[href*="/companycredits/"]) a'
        )

        # Filtra e remove "Empresas de produção" e "Production companies"
        empresas = [
            e.get_text(strip=True)
            for e in empresas_elementos
            if "Empresas de produção" not in e.get_text() and "Production companies" not in e.get_text()
        ]

        # Remove duplicatas e junta os nomes em uma string
        empresas = ", ".join(set(empresas)).rstrip(",") if empresas else "N/A"



        # País de origem
        pais_elementos = soup.select('li[data-testid="title-details-origin"] a')
        pais = list(set(p.get_text(strip=True) for p in pais_elementos))
        pais = ", ".join(pais) if pais else "N/A"

        # Duração do filme
        duracao_element = soup.select_one('li[data-testid="title-techspec_runtime"] div')
        duracao = duracao_element.get_text(strip=True) if duracao_element else "N/A"

        # Elenco (Corrigido!)
# Extraindo todos os nomes do elenco da seção de créditos completos
        elenco = [
            a.get_text(strip=True)
            for a in soup.find_all("a", class_=lambda c: c and "sc-cd7dc4b7-1 kVdWAO" in c)
        ]

        # Garante que há resultado
        elenco = elenco if elenco else ["N/A"]

        # Em formato de string, se preferir
        elenco_str = ", ".join(elenco)
        elenco = elenco_str
        


        return generos, avaliacao, poster, diretores, empresas, pais, duracao, elenco


    d_filmes = pd.read_csv(caminho_d_filmes, delimiter=';', dtype=str)
    d_filmes = d_filmes.head(2)  # Selecionando apenas os primeiros 5 filmes
    dados_filmes = []

    for _, row in d_filmes.iterrows():
        cpb_roe = row['CPB_ROE']
        titulo = row['TITULO_ORIGINAL']
        print(f"Processando: {titulo}")
        url, titulo_retornado = _buscar_filme_imdb(titulo)
        if url:
            generos, avaliacao, poster, diretores, empresas, pais, duracao, elenco= _extrair_info_filme(
                url)
            dados_filmes.append({
                "CPB_ROE": cpb_roe,
                "TITULO_ORIGINAL": titulo_retornado,
                "GENEROS": generos,
                "ELENCO": elenco,
                "AVALIACAO_IMDB": avaliacao,
                "LINK_CAPA": poster,
                "DIRETORES": diretores,
                "PAIS_ORIGEM": pais,
                "EMPRESAS_PRODUCAO": empresas.strip(","),
                "DURACAO": duracao,


                
            })
        else:
            print(f"Filme {titulo} não encontrado.")

    df_generos = pd.DataFrame(dados_filmes)

    # Traduzindo a coluna 'GENEROS' para português
    df_generos['GENEROS'] = df_generos['GENEROS'].apply(
        lambda texto: GoogleTranslator(source="en", target="pt").translate(texto))

    arquivo_saida = os.path.join(caminho_saida, "d_info_filmes.csv")
    df_generos.to_csv(arquivo_saida, sep=';', index=False, encoding='utf-8')
    print(f"d_genero salvo em: {arquivo_saida}")
    return df_generos



# Caminhos dos arquivos
caminho_d_filmes = r"data\processados\d_filmes.csv"
caminho_saida = r"data\processados"

# Executando a função
df_generos = d_info_filmes(caminho_d_filmes,caminho_saida)
df_generos

Processando: SONIC THE HEDGEHOG 3
Processando: O AUTO DA COMPADECIDA 2
d_genero salvo em: data\processados\d_info_filmes.csv


,CPB_ROE,TITULO_ORIGINAL,GENEROS,ELENCO,AVALIACAO_IMDB,LINK_CAPA,DIRETORES,PAIS_ORIGEM,EMPRESAS_PRODUCAO,DURACAO
0,E2400387600000,SONIC THE HEDGEHOG 3,"Aventura animal, épica de fantasia, aventura g...","Jim Carrey, Ben Schwartz, Keanu Reeves, Idris ...",6.9,https://m.media-amazon.com/images/M/MV5BODU3NT...,Jeff Fowler,"Japan, United States","Original Film, , Paramount Pictures, Sega Samm...",1hour50minutes
1,B2400470400000,O AUTO DA COMPADECIDA 2,"Comédia sombria, comédia escura, comédia","Matheus Nachtergaele, Selton Mello, Taís Araúj...",6.2,https://m.media-amazon.com/images/M/MV5BMWM3Y2...,"Guel Arraes, Flávia Lacerda",Brazil,"Conspiração Filmes, H2O Produções, Claro",1hour44minutes
